In [3]:
from tqdm import tqdm_notebook
import SimpleITK as sitk
import numpy as np
import re
from os.path import join
from os import listdir
from random import shuffle, sample

In [4]:
from matplotlib import pyplot as plt
# Define a function to plot a batch or list of image patches in a grid
def plot_image(images, images_per_row=8):
    
    fig, axs = plt.subplots(int(np.ceil(len(images)/images_per_row)), images_per_row)
    
    c = 0
    for ax_row in axs:
        for ax in ax_row:
            if c < len(images):
                ax.imshow(images[c])
            ax.axis('off')            
            c += 1
    plt.show()

In [27]:
dataDir = "/projects/0/ismi2018/FINALPROJECTS/BREAST_THOMOSYNTHESIS"

def listCaseIDs(dataRoot):
    cases = [f.replace(".gtrt","") for f in listdir(join(dataRoot,"gtrs")) if ".gtrt" in f]
    return cases

def getLabels(caseID,dataRoot):
    filename = join(dataRoot,"gtrs",caseID+".gtrt")
    annotation = open(filename,"r").read().split('\n\n')
    regex = "\[\n[\d* \d* \d*\n+]+"
    prog = re.compile(regex)
    for ann in annotation:
        if("contour" in ann):
            annRegion = prog.findall(ann)
            annRegion = [item for r in annRegion for item in r.split("\n")[1:]]
            annRegion = [[int(c) for c in cords.split()] for cords in annRegion if len(cords.split()) == 3]
            annRegion = np.asarray(annRegion)
        elif("points" in ann):
            annPoints = prog.findall(ann)
            annPoints = [item for r in annPoints for item in r.split("\n")[1:]]
            annPoints = [[int(c) for c in cords.split()] for cords in annPoints if len(cords.split()) == 3]
            annPoints = np.asarray(annPoints)
    return annRegion, annPoints

def makeMask(caseID,dataRoot,dims=None):
    anotationFileName = join(dataRoot,"gtrs",caseID+".gtrt")
    points = getPoints(anotationFileName,dataRoot)
    if(dims is None):
        dims = np.max(points,axis=0) + 1
    Mask = np.zeros(dims)
    Mask[points[:,1],points[:,2],points[:,2]] = 1.0
    return Mask

def loadScan(caseID,dataRoot):
    dataFolder = join(dataRoot,"dataset","t" + caseID)
    images = [f for f in listdir(dataFolder) if ".dcm" in f]
    images.sort(key=lambda x: float(x.replace(".dcm","")))
    
    scan = np.zeros(getDims(caseID,dataRoot))
    for i,f in enumerate(images):
        image = sitk.ReadImage ( join(dataFolder,f) )
        scan[:,:,i] = sitk.GetArrayFromImage(image)
    return scan
    
def getDims(caseID,dataRoot):
    dataFolder = join(dataRoot,"dataset","t" + caseID)
    images = [f for f in listdir(dataFolder) if ".dcm" in f]
    image = sitk.ReadImage ( join(dataFolder,images[0]) )
    dims = image.GetSize()
    dims = (dims[1],dims[0],len(images))
    return dims

def getregionMax(regions):
    r_x = regions[:,0]
    r_y = regions[:,1]
    return np.min(r_x), np.max(r_x), np.min(r_y), np.max(r_y)


def saveRegions(caseID, dataRoot, regions, points, boundingSize=50):
    cases = [f.replace(".dcm","") for f in listdir(join(dataRoot,"dataset", ''.join(['t',caseID]))) if ".dcm" in f]
    x_min, x_max, y_min, y_max = getregionMax(regions)
    for point in points:
        if str(point[2]) in cases:
            # Extract region
            image = sitk.ReadImage(join(dataRoot,"dataset", ''.join(['t',caseID]), ''.join([point[2],'.dcm'])))
            image = sitk.GetArrayFromImage(image)
            image = image[x_min:x_max, y_min:y_max]
            print(image.shape)

In [28]:
ids = listCaseIDs(dataDir)
print(ids)
regions, points = getLabels(ids[2],dataDir)
saveRegions(ids[0], dataDir, regions, points)



#Loop door de Z van points. Als er een calc aangegeven is, sla de regio op.


['0201074300mr', '0201077700cl', '0200714501cr', '0201079100cr', '0200526604ml', '0200904901cl', '0200893101cl', '0200701601mr', '0200893101ml', '0200687902mr', '0201073600mr', '0200963300ml', '0200671002cl', '0200526604cl', '0201082000cl', '0200926501cr', '0200953201cr', '0200895801mr', '0200693202mr', '0200963300cl', '0200951601ml', '0200917001cr', '0201078400mr', '0201082000ml', '0200414603cl', '0200677803cr', '0201082500mr', '0200890501cl', '0200693902mr', '0201073400cr', '0200890201mr', '0201077700ml', '0200536302cl', '0200960700cl', '0200953301cl', '0201076200cr', '0200693202cr', '0200967900ml', '0200951601cl', '0200710001cr', '0200953301ml', '0200677803mr', '0201073400mr', '0200921801cr', '0200890501ml', '0200962800cr', '0201078400cr', '0201073600cr', '0200710601ml', '0200486702mr', '0200710001mr', '0200641303cl', '0201075300ml', '0200888101cl', '0200917001mr', '0200671302cr', '0200916201ml', '0200697901ml', '0200713701cr', '0200952801mr', '0201079100mr', '0200713701mr', '020096

TypeError: sequence item 0: expected str instance, numpy.int64 found